In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
#collisions = pd.read_csv("/content/drive/My Drive/MIE368 Project - Group 15/Code/collisions_dataset/collisions_dataset_new/collisions_enriched.csv")
#speed_cameras = pd.read_csv("/content/drive/My Drive/MIE368 Project - Group 15/Code/speed_camera_dataset/speed_camera_clean_new/speed_cameras_clean.csv")

collisions = pd.read_csv("/content/drive/My Drive/UofT/Third Year/Fall/MIE368 Project - Group 15/Code/collisions_dataset/collisions_dataset_new/collisions_enriched.csv")
speed_cameras = pd.read_csv("/content/drive/My Drive/UofT/Third Year/Fall/MIE368 Project - Group 15/Code/speed_camera_dataset/speed_camera_clean_new/speed_cameras_clean.csv")

In [4]:
print("Collisions data:")
collisions.head()

Collisions data:


,date,hour,dow,lat,lon,severity,wx_precip_day,wx_precipitation,wx_rain,wx_snow,...,wx_avg_hourly_wind_speed,wx_max_relative_humidity,wx_avg_relative_humidity,wx_precip_amount_any,cam_nearest_m,cam_within_250m,cam_location,cam_status_clean,cam_ward_num,cam_fid
0,2014-01-01,17,Wednesday,43.701225,-79.377616,Property Damage Only,0,0.0,0.0,0.0,...,19.38,75.0,58.5,0.0,747.607580,0,Mount Pleasant Rd. Near Balliol St.,planned,12.0,171.0
1,2014-01-01,14,Wednesday,43.726091,-79.397589,Property Damage Only,0,0.0,0.0,0.0,...,19.38,75.0,58.5,0.0,307.110236,0,Ranleigh Ave. East of Yonge St.,active,15.0,80.0
2,2014-01-01,4,Wednesday,43.762676,-79.336644,Property Damage Only,0,0.0,0.0,0.0,...,19.38,75.0,58.5,0.0,169.482955,1,Fenelon Dr. Near Karen Rd.,active,16.0,87.0
3,2014-01-01,11,Wednesday,43.703234,-79.346615,Property Damage Only,0,0.0,0.0,0.0,...,19.38,75.0,58.5,0.0,187.001985,1,Thorncliffe Park Dr. Near 11 Thorncliffe Park Dr.,active,15.0,82.0
4,2014-01-01,1,Wednesday,43.650410,-79.378428,Property Damage Only,0,0.0,0.0,0.0,...,19.38,75.0,58.5,0.0,694.648506,0,Lower Jarvis St. South of The Esplanade,active,10.0,58.0


In [6]:
print("\nSpeed cameras data:")
speed_cameras[speed_cameras['status_clean'] == 'active']


Speed cameras data:


,lon,lat,status_clean,ward_num,location,fid
0,-79.567451,43.713609,active,1,Kipling Ave. North of Rexdale Blvd.,1
1,-79.550510,43.700973,active,1,St. Andrews Blvd. West of Islington Ave.,2
2,-79.561386,43.728553,active,1,Islington Ave. North of Fordwich Cres.,3
3,-79.597523,43.748744,active,1,Martin Grove Rd. South of Silverstone Dr.,4
4,-79.553404,43.722559,active,1,Golfdown Dr. East of Turpin Ave.,5
...,...,...,...,...,...,...
144,-79.219597,43.814072,active,25,Hupfield Trail Near Glanvil Cres.,145
145,-79.209388,43.797780,active,25,Murison Blvd. Near United Square,146
146,-79.133550,43.785117,active,25,East Ave. Near Maberley Crescent,147
177,-79.345589,43.732143,active,16,The Donway W. Near Southill Dr.,178


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# convert collisions to GeoDataFrame
collisions_gdf = gpd.GeoDataFrame(
    collisions,
    geometry=gpd.points_from_xy(collisions.lon, collisions.lat),
    crs="EPSG:4326"
).to_crs("EPSG:3857")

# convert cameras to GeoDataFrame
speed_gdf = gpd.GeoDataFrame(
    speed_cameras,
    geometry=gpd.points_from_xy(speed_cameras.lon, speed_cameras.lat),
    crs="EPSG:4326"
).to_crs("EPSG:3857")

In [ ]:
from shapely.geometry import box

collisions_gdf = collisions_gdf.to_crs(epsg=32617)
speed_gdf = speed_gdf.to_crs(epsg=32617)

# create a spatial grid (500m x 500m) to aggregate collisions
xmin, ymin, xmax, ymax = collisions_gdf.total_bounds
cell_size = 500  # in meters

cols = list(range(int(xmin), int(xmax) + cell_size, cell_size))
rows = list(range(int(ymin), int(ymax) + cell_size, cell_size))

grid_cells = []
for x in cols:
    for y in rows:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=collisions_gdf.crs)

# changed to .intersects(poly)
grid["collision_count"] = grid.geometry.apply(
    lambda poly: collisions_gdf.intersects(poly).sum()
)

grid["camera_count"] = grid.geometry.apply(
    lambda poly: speed_gdf.intersects(poly).sum()
)

In [ ]:
def mean_dist(poly):
    pts = collisions_gdf[collisions_gdf.intersects(poly)]
    return pts["cam_nearest_m"].mean() if len(pts) > 0 else 0

grid["mean_dist_to_camera"] = grid.geometry.apply(mean_dist)

In [ ]:
def mean_col(poly, col):
    pts = collisions_gdf[collisions_gdf.intersects(poly)]
    return pts[col].mean() if len(pts) > 0 else 0

grid["mean_precip"] = grid.geometry.apply(lambda p: mean_col(p, "wx_precipitation"))
grid["mean_snow"]   = grid.geometry.apply(lambda p: mean_col(p, "wx_snow"))

In [ ]:
# Downtown Toronto
downtown_poly = gpd.GeoSeries(
    [box(-79.42, 43.63, -79.34, 43.67)],
    crs="EPSG:4326"
).to_crs(32617)[0]

grid["downtown"] = grid.geometry.apply(
    lambda g: 1 if g.intersects(downtown_poly) else 0
)

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

model2 = smf.glm(
    formula="collision_count ~ camera_count + mean_dist_to_camera + mean_precip + mean_snow + downtown",
    data=grid,
    family=sm.families.NegativeBinomial()
).fit()

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [ ]:
print("\nMODEL 2 — Negative Binomial Regression")
print(model2.summary())


MODEL 2 — Negative Binomial Regression
                 Generalized Linear Model Regression Results                  
Dep. Variable:        collision_count   No. Observations:                 5270
Model:                            GLM   Df Residuals:                     5264
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18040.
Date:                Tue, 18 Nov 2025   Deviance:                       8687.3
Time:                        17:11:03   Pearson chi2:                 1.19e+04
No. Iterations:                    48   Pseudo R-squ. (CS):             0.9916
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------

In [ ]:
import math

coef_camera_count = model2.params['camera_count']
print("Coefficient for camera_count:", coef_camera_count)

irr_camera_count = math.exp(coef_camera_count)
print(f"IRR for camera_count: {irr_camera_count:.3f}") # Incidence Rate Ratio (IRR)

Coefficient for camera_count: 0.9537478535188455
IRR for camera_count: 2.595


In [ ]:
grid["lambda"] = model2.predict(grid)

## Model 2: Negative Binomial Regression Results

This model predicts the **count of collisions** within 500m × 500m spatial grid cells using the number of speed cameras (`camera_count`) in each cell.

### Interpretation:
- The **positive and significant coefficient** for `camera_count` indicates that grid cells with more cameras currently also have more collisions.
- The **IRR** is approximately 2.97, meaning the expected number of collisions **nearly triples** for each additional camera in a cell.
- **Important:** This does **not** mean cameras cause collisions. Rather, current cameras are placed in historically high-collision areas.

### Weather and Other Biases:
- **Mean Precipitation (`mean_precip`)** has a strong positive effect on collision counts. A **1-unit increase** in precipitation leads to a substantial **increase in collision frequency** (IRR of 2.611), indicating that **rainy conditions** contribute to more collisions.
- **Mean Snow (`mean_snow`)** has a negative relationship with collisions, but this is less impactful. The **IRR of -0.483** suggests that snow **reduces collision frequency** slightly, potentially because of reduced speeds during snowy weather.
- **Distance to Nearest Camera (`mean_dist_to_camera`)** is positive and significant, meaning that collisions are more likely in areas that are **farther from cameras**, which may indicate **gaps in camera coverage** or that cameras are **concentrated in high-risk areas**.
- **Downtown Flag (`downtown`)** has a positive coefficient (IRR of 1.148), meaning that **downtown areas** experience **more collisions** and are likely **overrepresented** in the model, which could skew results.

### Implications for Camera Placement:
- While cameras currently align with high-risk areas, the large IRR suggests that there are **other high-collision locations with few or no cameras**.
- This indicates opportunities to **optimize camera placement** to reduce collisions in under-covered areas.

### Coefficient for `camera_count`:
0.9537

### IRR for `camera_count`:
2.595

### Conclusion:
The model highlights that speed cameras are indeed placed in areas with higher collisions, but there may be **untapped potential** in other **high-risk areas**.
- The **IRR value of 2.595** points to a significant effect of camera placement on collision rates, suggesting a need for **strategic relocation** or **additional placement** of cameras in currently **under-served locations**.